# Noise (in PyQrack)

Qrack and PyQrack were designed for maximum performance on exact simulation of quantum circuits, hence no noise models are internally supported, yet. Further, PyQrack only supports "pure" quantum states, and not density matrix representation, for example. However, there are ways in which we can externally and stochastically apply common types of noise to a PyQrack simulation.

When using external stochastic noise gadgets, direct queries of exact quantum state and measurement sampling after a unitary preamble are **not** supported. The methods can still be called, but the pure state returned **cannot** be equivalent to a density matrix with noise, (i.e.: with partially mixed states, in effect,) and sampling operations will sample the happenstance probabilistic noise instead of the full stochastic profile of this noise. To simulate distributions with noise, we must run the circuits top-to-bottom for each sample, with probabilistic noise injection gadgets.

An obvious and simple type of noise injection gadget is for a probabilistic Pauli operator bit flip error.

In [1]:
import random

In [2]:
def pauli_bit_flip_noise_gadget(simulator, qubit, probability):
    if (not probability >= 1.) and ((probability <= 0.) or (random.uniform(0., 1.) >= probability)):
        # We avoid the bit flip error
        return

    # We apply a bit flip error
    axis = random.randint(0, 2)
    if axis == 0:
        simulator.x(qubit)
    elif axis == 1:
        simulator.y(qubit)
    else:
        simulator.z(qubit)

In [3]:
from collections import Counter
from pyqrack import QrackSimulator

qsim = QrackSimulator(1)

results = []
for i in range(0, 100):
    qsim.reset_all()
    pauli_bit_flip_noise_gadget(qsim, 0, 0.2)
    results.append(qsim.m(0))

print(Counter(results))

Device #0, Loaded binary from: /home/iamu/.qrack/qrack_ocl_dev_Intel(R)_UHD_Graphics_[0x9bc4].ir
Device #1, Loaded binary from: /home/iamu/.qrack/qrack_ocl_dev_NVIDIA_GeForce_RTX_3080_Laptop_GPU.ir
Counter({0: 82, 1: 18})


Depolarizing noise (as on a single qubit noise channel) partially replaces a qubit's pure quantum state with a maximally mixed state, according to a tunable noise severity parameter.

(**Warning**: This hypothetical approach to recreating stochastic depolarizing noise has not been vetted. This is purely Dan's first guess, as lead developer of Qrack. It does effectively fully depolarize for a `lam` parameter value of `1.`, though.)

In [4]:
import math
from pyqrack import Pauli

def depolarizing_1qb_noise_gadget(simulator, num_qubits, qubit, lam):
    if (lam <= 0.):
        return

    # Random azimuth around axis of measurement
    angleZ = random.uniform(0., 4. * math.pi)
    # Depolarization angle
    lamAngle = math.asin(lam)
    
    simulator.allocate_qubit(num_qubits)
    simulator.r(Pauli.PauliZ, angleZ, qubit)
    simulator.r(Pauli.PauliY, lamAngle, qubit)
    simulator.mcx([qubit], num_qubits)
    simulator.m(num_qubits)
    simulator.r(Pauli.PauliY, -lamAngle, qubit)
    simulator.r(Pauli.PauliZ, -angleZ, qubit)
    simulator.release(num_qubits)

In [5]:
qsim = QrackSimulator(1)

results = []
# This fixes the same random angle for 100 trials.
# Try varying these (fixed) angles and the depolarization parameter.
angleX = random.uniform(0., 4. * math.pi)
angleY = random.uniform(0., 4. * math.pi)
angleZ = random.uniform(0., 4. * math.pi)
for i in range(0, 100):
    qsim.reset_all()
    qsim.u(0, angleX, angleY, angleZ)
    depolarizing_1qb_noise_gadget(qsim, 1, 0, 0.2)
    results.append(qsim.m(0))

print(Counter(results))

Counter({0: 92, 1: 8})
